# Data Modeling with Cassandra

This Notebook has been submitted as part of the second project "Data Modeling with Cassandra" belonging to the Data Engineering Nanodegree by Udacity.

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [20]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [21]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [22]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [23]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Working with Apache Cassandra

The CSV file titled event_datafile_new.csv is located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the **event_datafile_new.csv** after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [24]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [25]:
# Create a Keyspace 
query = "CREATE KEYSPACE event WITH replication = {'class':'SimpleStrategy','replication_factor':1}"
session.execute(query)

AlreadyExists: Keyspace 'event' already exists

#### Set Keyspace

In [26]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('event')
except Exception as e:
    print(e)

# Task descriptions

Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Task 1: Get the artist, song title and song length from a sessionId and the item in session.

For this query, I have chosen a composite primary key (a partition key) consisting of the combination of sessionId and itemInSession. This way the partition key is unique and the items are partitioned and sorted in a way that makes this type of query easily accessible.


In [27]:
## Task 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1 = "SELECT artist, song, length FROM song_item_in_session WHERE sessionId = 338 AND itemInSession = 4"

# Creating the Table
queryc = "CREATE TABLE IF NOT EXISTS song_item_in_session \
(sessionId int, itemInSession int, artist text, firstName text, gender text, lastName text, length float,\
level text, location text, song text, userId int, PRIMARY KEY (sessionId, itemInSession))"
                    
try:
    session.execute(queryc)
except Exception as e:
    print(e)

In [28]:
# Populating the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_item_in_session (sessionId,  itemInSession, artist, firstName, gender, \
                lastName, length, level, location, song, userId)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[1], line[2], line[4], float(line[5]), line[6], line[7], line[9], int(line[10])))

In [29]:
## SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Task 2: Get the artist, song title and user name sorted by the itemInSession from a userId and sessionId.

For this query, I have chosen a composite primary key (a partition key) consisting of the combination of userId and sessionId and a clustering key consisting of the itemInSession.
This way the partition key is unique and the items are partitioned and sorted in a way that makes this type of query easily accessible.


In [30]:
## Task 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


query2 = "SELECT artist, song, firstName, lastName FROM song_playlist_session WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession ASC"

# Create Table
queryc = "CREATE TABLE IF NOT EXISTS song_playlist_session \
(userId int, sessionId int, itemInSession int, artist text, firstName text, gender text, lastName text, length float,\
level text, location text, song text, PRIMARY KEY ((userId, sessionId), itemInSession))\
WITH CLUSTERING ORDER BY (itemInSession ASC)"
                    
try:
    session.execute(queryc)
except Exception as e:
    print(e)
    
# Open file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, firstName, gender, \
                lastName, length, level, location, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[2], line[4], float(line[5]), line[6], line[7], line[9], ))

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Task 3: Get the user name who listened for a given song.

For this query, I have chosen a composite primary key (a partition key) consisting of the combination of song and userId.
This way the partition key is unique.

When a user listens to a song multiple times (i.e. with different sessionId-itemInSession pairs) it will overwrite the row (the partition key song-userId will be kept unique. This is OK with the query we are asking, since we are interested in figuring out who listened to a particular song, not if they did it several times.


In [31]:
## Task 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query3 = "SELECT firstName, lastName FROM users_per_song WHERE song = 'All Hands Against His Own'"

# Create Table
queryc = "CREATE TABLE IF NOT EXISTS users_per_song \
(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
                    
try:
    session.execute(queryc)
except Exception as e:
    print(e)
    
# Open file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the `query` variable
        query = "INSERT INTO users_per_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)                   

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [32]:
## Drop the table before closing out the sessions

queryd1 = "DROP TABLE IF EXISTS song_item_in_session"
try:
    rows = session.execute(queryd1)
except Exception as e:
    print(e)

queryd2 = "DROP TABLE IF EXISTS song_playlist_session"
try:
    rows = session.execute(queryd2)
except Exception as e:
    print(e)
    
queryd3 = "DROP TABLE IF EXISTS users_per_song"
try:
    rows = session.execute(queryd3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [33]:
session.shutdown()
cluster.shutdown()